[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/50--spark-multi-source/11--mongo--movies.ipynb)



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz -O spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

--2022-01-14 14:26:18--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M   254MB/s    in 0.9s    

2022-01-14 14:26:32 (254 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [2]:
!pip install -q pyspark findspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 17.4 MB/s 


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [4]:
!sudo apt install mongodb
!sudo service mongodb start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libyaml-cpp0.5v5 amd64 0.5.2-4ubuntu1 [150 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mongo-tools amd64 3.6.3-0ubunt

In [11]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection movies var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection films var/video_movies.bson

--2022-01-14 14:29:15--  https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson [following]
--2022-01-14 14:29:15--  https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343355 (335K) [application/octet-stream]
Saving to: ‘var/video_movies.bson’

var/video_movies.bs 10

In [12]:
pip install pymongo

In [13]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://localhost:27017')
db = client.datascience

In [15]:
pp(list(db.movies.aggregate([
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
    {"$limit": 2}
])))

[{'_id': ObjectId('5692a53024de1e0ce2dfdca5'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 7.6,
  'ratio': 22.0,
  'title': 'Au bout du monde'},
 {'_id': ObjectId('5692a47c24de1e0ce2dfdb63'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 8.1,
  'ratio': 22.0,
  'title': 'Everything Will Be Ok'}]


In [16]:
!wget https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1-assembly.jar -O /content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar
!wget https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.10/mongo-java-driver-3.12.10.jar -O /content/spark-3.1.2-bin-hadoop3.2/jars/mongo-java-driver-3.12.10.jar

--2022-01-14 14:30:07--  https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1-assembly.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788378 (2.7M) [application/java-archive]
Saving to: ‘/content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar’

/content/spark-3.1. 100%[===================>]   2.66M  --.-KB/s    in 0.05s   

2022-01-14 14:30:07 (52.6 MB/s) - ‘/content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar’ saved [2788378/2788378]

--2022-01-14 14:30:07--  https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.10/mongo-java-driver-3.12.10.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... c

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("local").\
        config("spark.mongodb.input.uri","mongodb://localhost:27017/datascience.movies").\
        getOrCreate()

sc = spark.sparkContext

In [18]:
spark

In [19]:
pipeline = [
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
]

In [20]:
import json

In [23]:
df = (spark.read.format("mongo")
  .option("uri","mongodb://127.0.0.1/datascience.movies")
  .option("pipeline", json.dumps(pipeline))
  .load())

In [24]:
df.show()

+--------------------+--------------------+------+-----------------+--------------------+
|                 _id|              awards|rating|            ratio|               title|
+--------------------+--------------------+------+-----------------+--------------------+
|{5692a47c24de1e0c...|{1.0, null, 22 wi...|   8.1|             22.0|Everything Will B...|
|{5692a53024de1e0c...|{1.0, null, 22 wi...|   7.6|             22.0|    Au bout du monde|
|{5692a3ce24de1e0c...|{1.0, null, 17 wi...|   4.8|             17.0|Oy Vey! My Son Is...|
|{5692a4dc24de1e0c...|{1.0, null, 11 wi...|   7.2|             11.0|     Filler ve Çimen|
|{5692a51724de1e0c...|{1.0, null, 8 win...|   7.3|              8.0|          I Am David|
|{5692a14624de1e0c...|{2.0, null, Nomin...|   8.3|              6.0|   Wild Strawberries|
|{5692a2b924de1e0c...|{1.0, null, 6 win...|   6.4|              6.0|           Eu Te Amo|
|{5692a15624de1e0c...|{1.0, null, Nomin...|   8.6|              6.0|It's a Wonderful ...|
|{5692a570

In [25]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- awards: struct (nullable = true)
 |    |-- nominations: double (nullable = true)
 |    |-- oscars: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- bestAnimatedFeature: string (nullable = true)
 |    |    |    |-- bestMusic: string (nullable = true)
 |    |    |    |-- bestPicture: string (nullable = true)
 |    |    |    |-- bestScreenplay: string (nullable = true)
 |    |    |    |-- bestSoundEditing: string (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- wins: double (nullable = true)
 |-- rating: double (nullable = true)
 |-- ratio: double (nullable = true)
 |-- title: string (nullable = true)



In [26]:
from pyspark.sql import functions as F

In [29]:
df.select(
    F.col('title'),
    F.col('awards.wins').alias('wins'),
    F.col('rating')
).show(5)

+--------------------+----+------+
|               title|wins|rating|
+--------------------+----+------+
|Everything Will B...|22.0|   8.1|
|    Au bout du monde|22.0|   7.6|
|Oy Vey! My Son Is...|17.0|   4.8|
|     Filler ve Çimen|11.0|   7.2|
|          I Am David| 8.0|   7.3|
+--------------------+----+------+
only showing top 5 rows



In [30]:
df.registerTempTable('movies')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [32]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [34]:
sqlContext.sql("Select * from movies").show(3)

+--------------------+--------------------+------+-----+--------------------+
|                 _id|              awards|rating|ratio|               title|
+--------------------+--------------------+------+-----+--------------------+
|{5692a47c24de1e0c...|{1.0, null, 22 wi...|   8.1| 22.0|Everything Will B...|
|{5692a53024de1e0c...|{1.0, null, 22 wi...|   7.6| 22.0|    Au bout du monde|
|{5692a3ce24de1e0c...|{1.0, null, 17 wi...|   4.8| 17.0|Oy Vey! My Son Is...|
+--------------------+--------------------+------+-----+--------------------+
only showing top 3 rows



In [36]:
sqlContext.sql("Select title, awards.wins as wins, rating from movies").show(3)

+--------------------+----+------+
|               title|wins|rating|
+--------------------+----+------+
|Everything Will B...|22.0|   8.1|
|    Au bout du monde|22.0|   7.6|
|Oy Vey! My Son Is...|17.0|   4.8|
+--------------------+----+------+
only showing top 3 rows



In [37]:
moviesDF = sqlContext.sql("Select title, awards.wins as wins, rating from movies")

Save it to external storage

In [40]:
moviesDF.write.csv('var/movies')

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
moviesDF.write.csv('drive/MyDrive/AA-movies')

In [43]:
sc.stop()